# 개체명 인식 (NER)

이 노트북은 [AI for Beginners Curriculum](http://aka.ms/ai-beginners)에서 제공됩니다.

이 예제에서는 [개체명 인식을 위한 주석 코퍼스](https://www.kaggle.com/datasets/abhinavwalia95/entity-annotated-corpus) 데이터셋을 사용하여 NER 모델을 학습하는 방법을 배워봅니다. 진행하기 전에 [ner_dataset.csv](https://www.kaggle.com/datasets/abhinavwalia95/entity-annotated-corpus?resource=download&select=ner_dataset.csv) 파일을 현재 디렉토리에 다운로드하세요.


In [62]:
import pandas as pd
from tensorflow import keras
import numpy as np

## 데이터셋 준비하기

데이터셋을 데이터프레임으로 읽는 것부터 시작하겠습니다. Pandas 사용법에 대해 더 알고 싶다면 [데이터 처리에 관한 강의](https://github.com/microsoft/Data-Science-For-Beginners/tree/main/2-Working-With-Data/07-python)를 [초보자를 위한 데이터 과학](http://aka.ms/datascience-beginners)에서 확인해보세요.


In [3]:
df = pd.read_csv('ner_dataset.csv',encoding='unicode-escape')
df.head()

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,NaN,of,IN,O
2,NaN,demonstrators,NNS,O
3,NaN,have,VBP,O
4,NaN,marched,VBN,O


고유 태그를 얻고 태그를 클래스 번호로 변환하는 데 사용할 수 있는 조회 사전을 만듭시다:


In [4]:
tags = df.Tag.unique()
tags

array(['O', 'B-geo', 'B-gpe', 'B-per', 'I-geo', 'B-org', 'I-org', 'B-tim',
       'B-art', 'I-art', 'I-per', 'I-gpe', 'I-tim', 'B-nat', 'B-eve',
       'I-eve', 'I-nat'], dtype=object)

In [8]:
id2tag = dict(enumerate(tags))
tag2id = { v : k for k,v in id2tag.items() }

id2tag[0]

'O'

이제 어휘에 대해서도 동일한 작업을 수행해야 합니다. 간단히 하기 위해 단어 빈도를 고려하지 않고 어휘를 만들 것입니다. 실제로는 Keras 벡터라이저를 사용하고 단어 수를 제한하는 것이 좋을 수 있습니다.


In [14]:
vocab = set(df['Word'].apply(lambda x: x.lower()))
id2word = { i+1 : v for i,v in enumerate(vocab) }
id2word[0] = '<UNK>'
vocab.add('<UNK>')
word2id = { v : k for k,v in id2word.items() }

우리는 훈련을 위한 문장 데이터셋을 만들어야 합니다. 원본 데이터셋을 반복하며 모든 개별 문장을 `X`(단어 목록)과 `Y`(토큰 목록)으로 분리합시다.


In [41]:
X,Y = [],[]
s,t = [],[]
for i,row in df[['Sentence #','Word','Tag']].iterrows():
    if pd.isna(row['Sentence #']):
        s.append(row['Word'])
        t.append(row['Tag'])
    else:
        if len(s)>0:
            X.append(s)
            Y.append(t)
        s,t = [row['Word']],[row['Tag']]
X.append(s)
Y.append(t)


In [93]:
def vectorize(seq):
    return [word2id[x.lower()] for x in seq]

def tagify(seq):
    return [tag2id[x] for x in seq]

Xv = list(map(vectorize,X))
Yv = list(map(tagify,Y))

Xv[0], Yv[0]

([10386,
  23515,
  4134,
  29620,
  7954,
  13583,
  21193,
  12222,
  27322,
  18258,
  5815,
  15880,
  5355,
  25242,
  31327,
  18258,
  27067,
  23515,
  26444,
  14412,
  358,
  26551,
  5011,
  30558],
 [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0])

문장의 간단함을 위해 모든 문장을 최대 길이까지 0 토큰으로 패딩할 것입니다. 실제로는 더 영리한 전략을 사용하여 하나의 미니배치 내에서만 시퀀스를 패딩할 수 있습니다.


In [51]:
X_data = keras.preprocessing.sequence.pad_sequences(Xv,padding='post')
Y_data = keras.preprocessing.sequence.pad_sequences(Yv,padding='post')

## 토큰 분류 네트워크 정의

토큰 분류를 위해 이중 레이어 양방향 LSTM 네트워크를 사용할 것입니다. 마지막 LSTM 레이어의 각 출력에 밀집 분류기를 적용하기 위해, `TimeDistributed` 구조를 사용할 것입니다. 이 구조는 LSTM의 각 단계에서 동일한 밀집 레이어를 각 출력에 복제합니다:


In [94]:
maxlen = X_data.shape[1]
vocab_size = len(vocab)
num_tags = len(tags)
model = keras.models.Sequential([
    keras.layers.Embedding(vocab_size, 300, input_length=maxlen),
    keras.layers.Bidirectional(keras.layers.LSTM(units=100, activation='tanh', return_sequences=True)),
    keras.layers.Bidirectional(keras.layers.LSTM(units=100, activation='tanh', return_sequences=True)),
    keras.layers.TimeDistributed(keras.layers.Dense(num_tags, activation='softmax'))
])
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 104, 300)          9545400   
                                                                 
 bidirectional_6 (Bidirectio  (None, 104, 200)         320800    
 nal)                                                            
                                                                 
 bidirectional_7 (Bidirectio  (None, 104, 200)         240800    
 nal)                                                            
                                                                 
 time_distributed_3 (TimeDis  (None, 104, 17)          3417      
 tributed)                                                       
                                                                 
Total params: 10,110,417
Trainable params: 10,110,417
Non-trainable params: 0
__________________________________________

여기서 우리는 데이터셋에 대해 `maxlen`을 명시적으로 지정하고 있습니다. 네트워크가 가변 길이 시퀀스를 처리할 수 있도록 하려면 네트워크를 정의할 때 조금 더 신중해야 합니다.

이제 모델을 훈련시켜 봅시다. 속도를 위해 한 에포크만 훈련시키겠지만, 더 오랜 시간 동안 훈련을 시도해볼 수도 있습니다. 또한, 데이터셋의 일부를 훈련 데이터셋으로 분리하여 검증 정확도를 관찰하는 것도 고려해볼 수 있습니다.


In [57]:
model.fit(X_data,Y_data)

1499/1499 [==============================] - 740s 488ms/step - loss: 0.0667 - acc: 0.9841


## 결과 테스트하기

이제 샘플 문장에서 우리의 엔터티 인식 모델이 어떻게 작동하는지 확인해봅시다:


In [91]:
sent = 'John Smith went to Paris to attend a conference in cancer development institute'
words = sent.lower().split()
v = keras.preprocessing.sequence.pad_sequences([[word2id[x] for x in words]],padding='post',maxlen=maxlen)
res = model(v)[0]

In [92]:
r = np.argmax(res.numpy(),axis=1)
for i,w in zip(r,words):
    print(f"{w} -> {id2tag[i]}")

john -> B-per
smith -> I-per
went -> O
to -> O
paris -> B-geo
to -> O
attend -> O
a -> O
conference -> O
in -> O
cancer -> B-org
development -> I-org
institute -> I-org


## 주요 내용

간단한 LSTM 모델만으로도 NER에서 괜찮은 결과를 얻을 수 있습니다. 하지만 훨씬 더 나은 결과를 얻고 싶다면 BERT와 같은 대규모 사전 학습된 언어 모델을 사용하는 것이 좋습니다. Huggingface Transformers 라이브러리를 사용하여 BERT를 NER에 맞게 학습시키는 방법은 [여기](https://huggingface.co/course/chapter7/2?fw=pt)에서 설명되어 있습니다.



---

**면책 조항**:  
이 문서는 AI 번역 서비스 [Co-op Translator](https://github.com/Azure/co-op-translator)를 사용하여 번역되었습니다. 정확성을 위해 최선을 다하고 있으나, 자동 번역에는 오류나 부정확성이 포함될 수 있습니다. 원본 문서의 원어 버전을 권위 있는 출처로 간주해야 합니다. 중요한 정보의 경우, 전문적인 인간 번역을 권장합니다. 이 번역 사용으로 인해 발생하는 오해나 잘못된 해석에 대해 책임을 지지 않습니다.
